In [1]:
from __future__ import division
import sys
import os
import time
import glob
import PIL.Image as Image
import numpy as np

import theano
import theano.tensor as T

import lasagne
#import lasagne.layers.dnn

import Cae

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 750 Ti (CNMeM is disabled, cuDNN 5110)


In [2]:
def im2ar(x):
    y = [[[0]*len(x)]*len(x)]*3
    y[0] = x[:,:,0]
    y[1] = x[:,:,1]
    y[2] = x[:,:,2]
    return y

def ar2im(x):
    y = [[[0]*3]*len(x)]*len(x)
    y[:,:,0] = x[0,:,:] 
    y[:,:,1] = x[1,:,:]
    y[:,:,2] = x[2,:,:]
    return y

In [3]:
def load_dataset_mscoco():
    #Path
    mscoco="/home/myuser/Documents/Lasagne/inpainting"
    split="train2014"
    data_path = os.path.join(mscoco, split)
    imgs = glob.glob(data_path + "/*.jpg")
    
    X_train = []
    y_train = []
            
    for i, img_path in enumerate(imgs):
        img = Image.open(img_path)
        img_array = np.divide(np.array(img,dtype='float32'),255)

        if len(img_array.shape) == 3:
            temp = np.copy(img_array)
            input = np.copy(img_array)
            input[16:48, 16:48,:] = 0
            target = img_array[16:48, 16:48,:]
        else:
            input[:,:,0] = np.copy(img_array)
            input[:,:,1] = np.copy(img_array)
            input[:,:,2] = np.copy(img_array)
            target = input[16:48, 16:48,:]
            input[16:48, 16:48,:] = 0
        
        X_train.append(im2ar(input))
        y_train.append(im2ar(target))
    
    split="val2014"
    data_path = os.path.join(mscoco, split)
    imgs = glob.glob(data_path + "/*.jpg")
    
    X_val = []
    y_val = []
            
    for i, img_path in enumerate(imgs):
        img = Image.open(img_path)
        img_array = np.divide(np.array(img,dtype='float32'),255)

        if len(img_array.shape) == 3:
            input = np.copy(img_array)
            input[16:48, 16:48,:] = 0
            target = img_array[16:48, 16:48,:]
        else:
            input[:,:,0] = np.copy(img_array)
            input[:,:,1] = np.copy(img_array)
            input[:,:,2] = np.copy(img_array)
            target = input[16:48, 16:48,:]
            input[16:48, 16:48,:] = 0
        
        X_val.append(im2ar(input))
        y_val.append(im2ar(target))
    
    # We reserve the last 10000 training examples for testing.
    X_val, X_test = X_val[:-10000], X_val[-10000:]
    y_val, y_test = y_val[:-10000], y_val[-10000:]
    
    return (np.array(X_train),np.array(y_train),np.array(X_val),np.array(y_val),np.array(X_test),np.array(y_test))

In [4]:
# Batch iterator
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [5]:
# Main
def main():
    
    # Hyper Params
    num_epochs = 100
    learning_rate = 0.01
    momentum = 0.975
    batchsize = 200
    
    # Load the dataset
    print("Loading data...")
    X_train, y_train, X_val, y_val, X_test, y_test = load_dataset_mscoco()
    
    # Prepare Theano variables for inputs and targets
    inputVar = T.tensor4('inputs')
    target_var = T.tensor4('targets')

    # Build Network
    print("Building model and compiling functions...")
    network = Cae.build(inputVar)

    # Training Loss expression
    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.squared_error(prediction, target_var)
    loss = loss.mean()
    # Add regularization lasagne.regularization.

    # Update expressions 
    # Stochastic Gradient Descent (SGD) with Nesterov momentum
    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate, momentum)

    # Test Loss expression
    # 'Deterministic = True' disables droupout
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.squared_error(test_prediction,target_var)
    test_loss = test_loss.mean()

    # Train Function
    train_fn = theano.function([inputVar, target_var], loss, updates=updates)

    # Test Function
    val_fn = theano.function([inputVar, target_var], test_loss)
    # Training Loop
    print("Starting training...")
    # We iterate over epochs
    for epoch in range(num_epochs):
        # Full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # Full pass over the validation data
        val_err = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, batchsize, shuffle=False):
            inputs, targets = batch
            err = val_fn(inputs, targets)
            val_err += err
            val_batches += 1

        # Print the results for this epoch
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))

    # Print the test error
    test_err = 0
    test_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batchsize, shuffle=False):
        inputs, targets = batch
        err = val_fn(inputs, targets)
        test_err += err
        test_batches += 1
    print("Final results:")
    print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))


    # Save model
    np.savez('cae-100e.npz', *lasagne.layers.get_all_param_values(network))
        

In [6]:
if __name__ == '__main__':
        main()

Loading data...
Building model and compiling functions...
Starting training...
Epoch 1 of 100 took 193.952s
  training loss:		0.071211
  validation loss:		0.065530
Epoch 2 of 100 took 193.797s
  training loss:		0.059697
  validation loss:		0.051746
Epoch 3 of 100 took 193.802s
  training loss:		0.048448
  validation loss:		0.046077
Epoch 4 of 100 took 193.854s
  training loss:		0.044976
  validation loss:		0.043772
Epoch 5 of 100 took 194.131s
  training loss:		0.043242
  validation loss:		0.042396
Epoch 6 of 100 took 194.359s
  training loss:		0.042113
  validation loss:		0.041624
Epoch 7 of 100 took 193.890s
  training loss:		0.041356
  validation loss:		0.040935
Epoch 8 of 100 took 193.850s
  training loss:		0.040816
  validation loss:		0.040460
Epoch 9 of 100 took 196.376s
  training loss:		0.040387
  validation loss:		0.040091
Epoch 10 of 100 took 194.938s
  training loss:		0.040018
  validation loss:		0.039755
Epoch 11 of 100 took 194.022s
  training loss:		0.039579
  validation 